In [1]:
import mxnet as mx
from mxnet import nd 
from mxnet import autogradgrad

In [2]:
x = nd.array([[1,2], [3,4]])
x


[[1. 2.]
 [3. 4.]]
<NDArray 2x2 @cpu(0)>

In [7]:
x.attach_grad()

In [8]:
def f(x):
    return x * x**2

In [9]:
with autograd.record():
    y = f(x)

In [10]:
x,y

(
 [[1. 2.]
  [3. 4.]]
 <NDArray 2x2 @cpu(0)>, 
 [[ 1.  8.]
  [27. 64.]]
 <NDArray 2x2 @cpu(0)>)

In [11]:
y.backward()

In [12]:
x, x.grad

(
 [[1. 2.]
  [3. 4.]]
 <NDArray 2x2 @cpu(0)>, 
 [[ 3. 12.]
  [27. 48.]]
 <NDArray 2x2 @cpu(0)>)

In [13]:
def f(x):
    x = x**2
    while x.norm().asscalar() < 1000:
        x = x*2
    if x.sum().asscalar() >= 0:
        y = x[0]
    else:
        y = x[1]
    return y

In [14]:
x = nd.random.uniform(-1,1,shape=2)
x


[0.09762704 0.18568921]
<NDArray 2 @cpu(0)>

In [15]:
x.attach_grad()
with autograd.record():
    y = f(x)
y.backward()

In [16]:
from mxnet import nd, autograd, optimizer, gluon

In [18]:
net = gluon.nn.Dense(1)


In [19]:
net.initialize()

In [20]:
batch_size = 8
x = nd.random.uniform(shape=(batch_size,4))
y = nd.random.uniform(shape=(batch_size,))

In [21]:
loss = gluon.loss.L2Loss()
def forword_backword():
    with autograd.record():
        l = loss(net(x),y)
    l.backward()

In [22]:
forword_backword()

In [24]:
trainer = gluon.Trainer(net.collect_params(),
                       optimizer='sgd', optimizer_params={'learning_rate':0.001})

In [25]:
c_w = net.weight.data().copy()
print(c_w)


[[ 0.0418822   0.0421275  -0.00539289  0.00286685]]
<NDArray 1x4 @cpu(0)>


In [26]:
print(c_w - net.weight.grad() *1 /8)


[[0.44725198 0.48446643 0.30377847 0.37031227]]
<NDArray 1x4 @cpu(0)>


In [27]:
trainer.learning_rate

0.001

In [28]:
trainer.set_learning_rate(0.01)
trainer.learning_rate

0.01

In [29]:
from matplotlib.pyplot import imshow

In [30]:
data = gluon.data.dataset.ArrayDataset(x,y)

In [31]:
sample = data[4]
sample

(
 [0.3834415  0.47766513 0.79172504 0.8121687 ]
 <NDArray 4 @cpu(0)>, 0.87001216)

In [35]:
def trasform_fn(data,  label):
    data = data.astype('float32') /255
    return data, label

In [36]:
train_data = gluon.data.vision.datasets.MNIST(train=True).transform(trasform_fn)
valid_data = gluon.data.vision.datasets.MNIST(train=False).transform(trasform_fn)

In [37]:
sample_img = train_data[19][0]
nd.max(sample_img)


[1.]
<NDArray 1 @cpu(0)>

In [38]:
from mxnet.gluon.data.vision import transforms

In [39]:
mean, std= (0.1307,),(0.3081,)

normalize = transforms.Normalize(mean, std)
train_data = train_data.transform_first(normalize)
print(train_data)

In [40]:
tratrain_dataset = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize(mean, std)])

In [41]:
tranform_dataset = gluon.data.vision.datasets.MNIST(train=True).transform(tratrain_dataset)

In [43]:
data_loader = gluon.data.DataLoader(data, batch_size=5, last_batch='keep')

for x,y in data_loader:
    print(x.shape, y.shape)

(5, 4) (5,)
(3, 4) (3,)


In [61]:
transform = transforms.Compose([transforms.RandomFlipTopBottom(),
                               transforms.RandomFlipLeftRight()])

In [63]:
train_data =  gluon.data.vision.datasets.MNIST(train=True)

In [64]:
train_dataloader = gluon.data.DataLoader(train_data.transform_first(transform), batch_size=3, shuffle=True)

In [65]:
train_dataloader